In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [589 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,185 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://ale-work-123.s3.us-west-1.amazonaws.com/heart_statlog_cleveland_hungary_final_clean.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("heart_statlog_cleveland_hungary_final_clean.csv"), sep=",", header=True)

df.show()

+---+---+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
|age|sex|chest_pain_type|resting_bp_s|cholesterol|fasting_blood_sugar|resting_ecg|max_heart_rate|exercise_angina|oldpeak|ST_slope|target|
+---+---+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
| 40|  1|              2|         140|        289|                  0|          0|           172|              0|    0.0|       1|     0|
| 49|  0|              3|         160|        180|                  0|          0|           156|              0|    1.0|       2|     1|
| 37|  1|              2|         130|        283|                  0|          1|            98|              0|    0.0|       1|     0|
| 48|  0|              4|         138|        214|                  0|          0|           108|              1|    1.5|       2|     1|
| 54|  1|              3|         

In [4]:
df.createOrReplaceTempView('heart_disease')

In [5]:
def run_sql_query(query):
    spark.sql(query).show()

In [6]:
## Type of chest pain compared to their cholesterol levels and if they have heart disease or not
run_sql_query("""
    select chest_pain_type as chest_pain_type, cholesterol as cholesterol, target as heart_disease
    from heart_disease
    where cholesterol != 0
    order by chest_pain_type
""")

+---------------+-----------+-------------+
|chest_pain_type|cholesterol|heart_disease|
+---------------+-----------+-------------+
|              1|        291|            1|
|              1|        273|            1|
|              1|        160|            0|
|              1|        270|            1|
|              1|        252|            0|
|              1|        223|            0|
|              1|        258|            1|
|              1|        272|            1|
|              1|        200|            1|
|              1|        308|            0|
|              1|        181|            0|
|              1|        237|            0|
|              1|        216|            1|
|              1|        171|            0|
|              1|        139|            0|
|              1|        156|            1|
|              1|        234|            1|
|              1|        249|            0|
|              1|        211|            0|
|              1|        199|   

In [7]:
## Individuals with their resting ecg and chest pain types and whether they have heart disease or not
run_sql_query("""
    select chest_pain_type, resting_ecg, target as heart_disease
    from heart_disease
    order by heart_disease
""")

+---------------+-----------+-------------+
|chest_pain_type|resting_ecg|heart_disease|
+---------------+-----------+-------------+
|              2|          0|            0|
|              3|          0|            0|
|              2|          0|            0|
|              4|          1|            0|
|              2|          0|            0|
|              3|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              3|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              1|          0|            0|
|              2|          1|            0|
|              2|          1|            0|
|              2|          1|            0|
|              3|          0|   

In [25]:
## The average resting blood pressure and cholesterol based on age and sex
run_sql_query("""
    select age, sex, round(avg(resting_bp_s), 2) as avg_resting_bp_s, round(avg(cholesterol), 2) as cholesterol
    from heart_disease
    group by age, sex
    order by age
""")

+---+---+----------------+-----------+
|age|sex|avg_resting_bp_s|cholesterol|
+---+---+----------------+-----------+
| 28|  1|           130.0|      132.0|
| 29|  1|           130.0|     236.67|
| 30|  0|           170.0|      237.0|
| 31|  1|           120.0|      270.0|
| 31|  0|           100.0|      219.0|
| 32|  0|           105.0|      198.0|
| 32|  1|           112.0|      252.0|
| 33|  1|           120.0|      298.0|
| 33|  0|           100.0|      246.0|
| 34|  0|           124.0|      185.5|
| 34|  1|           124.2|      154.4|
| 35|  0|          132.67|      170.0|
| 35|  1|          123.88|     207.75|
| 36|  1|          123.67|     190.33|
| 37|  0|           125.0|     214.75|
| 37|  1|          128.29|     244.57|
| 38|  1|          121.92|     159.15|
| 38|  0|          111.67|      91.67|
| 39|  0|           114.0|     200.33|
| 39|  1|          130.67|     248.92|
+---+---+----------------+-----------+
only showing top 20 rows



In [26]:
## Individuals marked with high cholesterol and whether they have heart disease or not
run_sql_query("""
    select cholesterol, target as heart_disease
    from heart_disease
    where cholesterol >= 200
""")

+-----------+-------------+
|cholesterol|heart_disease|
+-----------+-------------+
|        289|            0|
|        283|            0|
|        214|            1|
|        339|            0|
|        237|            0|
|        208|            0|
|        207|            1|
|        284|            0|
|        211|            0|
|        204|            0|
|        234|            1|
|        211|            0|
|        273|            0|
|        201|            0|
|        248|            1|
|        267|            1|
|        223|            0|
|        201|            0|
|        288|            1|
|        215|            0|
+-----------+-------------+
only showing top 20 rows



In [39]:
## Average cholesterol based on their sex and if they have heart disease or not
run_sql_query("""
    select sex, target as heart_disease, round(avg(cholesterol), 2)
    from heart_disease
    where cholesterol != 0
    group by sex, target
    order by target ASC
""")

+---+-------------+--------------------------+
|sex|heart_disease|round(avg(cholesterol), 2)|
+---+-------------+--------------------------+
|  0|            0|                    249.18|
|  1|            0|                    232.81|
|  0|            1|                    279.18|
|  1|            1|                     247.5|
+---+-------------+--------------------------+



In [35]:
## Average cholesterol and resting blood pressure for individuals with heart disease based on sex
run_sql_query("""
    select sex, round(avg(cholesterol), 2) as avg_cholesterol, round(avg(resting_bp_s), 2) as resting_bp_s, count(target) as with_heart_disease
    from heart_disease
    where cholesterol != 0 and resting_bp_s != 0 and target != 0
    group by sex
""")

+---+---------------+------------+------------------+
|sex|avg_cholesterol|resting_bp_s|with_heart_disease|
+---+---------------+------------+------------------+
|  0|         279.18|      143.38|                40|
|  1|          247.5|      135.24|               316|
+---+---------------+------------+------------------+



In [36]:
## Average cholesterol and resting blood pressure for individuals without heart disease based on sex
run_sql_query("""
    select sex, round(avg(cholesterol), 2) as avg_cholesterol, round(avg(resting_bp_s), 2) as resting_bp_s, count(target) as without_heart_disease
    from heart_disease
    where cholesterol != 0 and resting_bp_s != 0 and target != 1
    group by sex
""")

+---+---------------+------------+---------------------+
|sex|avg_cholesterol|resting_bp_s|without_heart_disease|
+---+---------------+------------+---------------------+
|  0|         249.18|      128.77|                  142|
|  1|         232.81|      130.96|                  248|
+---+---------------+------------+---------------------+



In [12]:
## Average Cholesterol and Average Resting blood pressure based on those with or without heart disease
run_sql_query("""
    select target as heart_disease, round(avg(cholesterol), 2) as avg_cholesterol, round(avg(resting_bp_s), 2) as resting_bp_s
    from heart_disease
    where cholesterol != 0 or resting_bp_s = 0
    group by target
""")

+-------------+---------------+------------+
|heart_disease|avg_cholesterol|resting_bp_s|
+-------------+---------------+------------+
|            0|         238.77|      130.16|
|            1|         250.36|      135.77|
+-------------+---------------+------------+



In [33]:
## Number of individuals with each chest pain type and if they have heart disease or not
run_sql_query("""
    select target as heart_disease, chest_pain_type, count(chest_pain_type) as chest_pain_type_count
    from heart_disease
    group by target, chest_pain_type
    order by chest_pain_type, target ASC
""")

+-------------+---------------+---------------------+
|heart_disease|chest_pain_type|chest_pain_type_count|
+-------------+---------------+---------------------+
|            0|              1|                   26|
|            1|              1|                   20|
|            0|              2|                  149|
|            1|              2|                   24|
|            0|              3|                  131|
|            1|              3|                   72|
|            0|              4|                  104|
|            1|              4|                  392|
+-------------+---------------+---------------------+



In [14]:
## People with or without exercise angina and their average cholesterol
run_sql_query("""
    select exercise_angina, count(exercise_angina), round(avg(cholesterol), 2) as avg_choleseterol
    from heart_disease
    where cholesterol != 0
    group by exercise_angina
""")

+---------------+----------------------+----------------+
|exercise_angina|count(exercise_angina)|avg_choleseterol|
+---------------+----------------------+----------------+
|              0|                   459|          240.58|
|              1|                   287|          251.12|
+---------------+----------------------+----------------+



In [15]:
## Count the number of individuals with heart disease based on their sex
run_sql_query("""
    select sex as sex, count(target) as heart_disease
    from heart_disease
    where target == 1
    group by sex
""")

+---+-------------+
|sex|heart_disease|
+---+-------------+
|  0|           50|
|  1|          458|
+---+-------------+



In [16]:
## Count the number of individuals without heart disease based on their sex
run_sql_query("""
    select sex as sex, count(target) as heart_disease
    from heart_disease
    where target == 0
    group by sex
""")

+---+-------------+
|sex|heart_disease|
+---+-------------+
|  0|          143|
|  1|          267|
+---+-------------+



In [18]:
## The cholesterol, resting blood pressure, whether they have exercise angina or not, and whether they have heart disease or not in individuals with the typical angina chest pain type
run_sql_query("""
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 1 and cholesterol != 0 and resting_bp_s != 0
    order by target
""")

+---------------+-----------+------------+---------------+------+
|chest_pain_type|cholesterol|resting_bp_s|exercise_angina|target|
+---------------+-----------+------------+---------------+------+
|              1|        223|         100|              0|     0|
|              1|        160|         120|              0|     0|
|              1|        193|         160|              0|     0|
|              1|        308|         130|              0|     0|
|              1|        249|         110|              0|     0|
|              1|        237|         170|              0|     0|
|              1|        171|         120|              0|     0|
|              1|        252|         140|              0|     0|
|              1|        181|         139|              0|     0|
|              1|        139|         135|              0|     0|
|              1|        211|         110|              1|     0|
|              1|        199|         140|              1|     0|
|         

In [19]:
## The cholesterol, resting blood pressure, whether they have exercise angina or not, and whether they have heart disease or not in individuals with the atypical angina chest pain type
run_sql_query("""
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 2 and cholesterol != 0 and resting_bp_s != 0
    order by target
""")

+---------------+-----------+------------+---------------+------+
|chest_pain_type|cholesterol|resting_bp_s|exercise_angina|target|
+---------------+-----------+------------+---------------+------+
|              2|        264|         150|              0|     0|
|              2|        241|         190|              0|     0|
|              2|        216|         140|              0|     0|
|              2|        283|         130|              0|     0|
|              2|        224|         140|              0|     0|
|              2|        208|         110|              0|     0|
|              2|        245|         130|              0|     0|
|              2|        194|         160|              0|     0|
|              2|        214|         150|              0|     0|
|              2|        253|         100|              0|     0|
|              2|        201|         120|              0|     0|
|              2|        202|         110|              0|     0|
|         

In [20]:
## The cholesterol, resting blood pressure, whether they have exercise angina or not, and whether they have heart disease or not in individuals with the non-anginal pain chest pain type
run_sql_query("""
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 3 and cholesterol != 0 and resting_bp_s != 0
    order by target
""")

+---------------+-----------+------------+---------------+------+
|chest_pain_type|cholesterol|resting_bp_s|exercise_angina|target|
+---------------+-----------+------------+---------------+------+
|              3|        272|         125|              0|     0|
|              3|        235|         130|              0|     0|
|              3|        228|         120|              1|     0|
|              3|        195|         150|              0|     0|
|              3|        292|         145|              0|     0|
|              3|        211|         130|              0|     0|
|              3|        179|         140|              0|     0|
|              3|        215|         130|              0|     0|
|              3|        235|         140|              0|     0|
|              3|        194|         130|              0|     0|
|              3|        200|         150|              0|     0|
|              3|        192|         135|              0|     0|
|         

In [21]:
## The cholesterol, resting blood pressure, whether they have exercise angina or not, and whether they have heart disease or not in individuals with the asymptomatic chest pain type
run_sql_query("""
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 4 and cholesterol != 0 and resting_bp_s != 0
    order by target
""")

+---------------+-----------+------------+---------------+------+
|chest_pain_type|cholesterol|resting_bp_s|exercise_angina|target|
+---------------+-----------+------------+---------------+------+
|              4|        297|         132|              0|     0|
|              4|        238|         110|              1|     0|
|              4|        412|         150|              0|     0|
|              4|        254|         120|              0|     0|
|              4|        238|         130|              0|     0|
|              4|        129|         140|              0|     0|
|              4|        227|         150|              1|     0|
|              4|        276|         140|              1|     0|
|              4|        180|         130|              1|     0|
|              4|        240|         110|              0|     0|
|              4|        224|         140|              0|     0|
|              4|        163|         108|              0|     0|
|         

In [40]:
## The average cholesterol and average age of people by chest pain type and sex
run_sql_query("""
    select chest_pain_type, sex, round(avg(cholesterol), 2), round(avg(age), 3)
    from heart_disease
    where cholesterol != 0
    group by chest_pain_type, sex
    order by chest_pain_type, sex
""")

+---------------+---+--------------------------+------------------+
|chest_pain_type|sex|round(avg(cholesterol), 2)|round(avg(age), 3)|
+---------------+---+--------------------------+------------------+
|              1|  0|                    234.33|            53.333|
|              1|  1|                    231.75|            55.313|
|              2|  0|                    253.58|            49.356|
|              2|  1|                    236.97|            48.495|
|              3|  0|                    250.33|             52.75|
|              3|  1|                    231.31|            52.376|
|              4|  0|                    265.55|            54.274|
|              4|  1|                    247.12|             54.75|
+---------------+---+--------------------------+------------------+

